In [2]:
!pip install textblob

from textblob import TextBlob

import pandas as pd

import requests

import json

In [7]:
#The 2019 MLB Int Class

mlb_json_id_url = 'http://m.mlb.com/gen/players/prospects/2019/playerProspects.json'
response = requests.get(mlb_json_id_url)
response.status_code
all_json = response.json()
int_prospect_list_2019 = all_json['prospect_players']['int']
int_player_ids_2019 = []
int_player_ranks_2019 = []
int_player_first_name_2019 = []
int_player_last_name_2019 = []
int_player_position_2019 = []
for i in int_prospect_list_2019:
    int_player_id_2019 = str(i['player_id'])
    int_player_ids_2019.append(int_player_id_2019)                      
    int_rank_2019 = i["rank"]
    int_player_ranks_2019.append(int_rank_2019)
    int_first_name_2019 = i["player_first_name"]
    int_player_first_name_2019.append(int_first_name_2019)
    int_last_name_2019 = i["player_last_name"]
    int_player_last_name_2019.append(int_last_name_2019)
    int_position_2019 = i["position"]
    int_player_position_2019.append(int_position_2019)
    int_player_id_data_2019 = {'First Name' : int_player_first_name_2019, 'Last Name' : int_player_last_name_2019, 'Player ID' : int_player_ids_2019, 'Position' : int_player_position_2019, 'Player Rank' : int_player_ranks_2019}

int_player_id_dataframe_2019 = pd.DataFrame(int_player_id_data_2019)


###Part 3: Scraping every Top100 report into dataframes
pipeline_url = "http://m.mlb.com/gen/players/prospects/2019/"
int_prospect_url_list_2019 = []
link_status_2019 = []
#making links
for i in int_player_ids_2019:
    prospect_url = pipeline_url + i + '.json'
    int_prospect_url_list_2019.append(prospect_url)

#testing links
for i in int_prospect_url_list_2019:    
    response = requests.get(i)
    if response.status_code == 200:
        link = 'Link is working'
    else:
        link = 'Link does not work'
    link_status_2019.append(link)
    
#making initial list of player scout-reports/cards
int_card_list_2019 = []
for i in int_prospect_url_list_2019:    
    response = requests.get(i)
    int_player_card_framed_2019 = response.json()
    int_player_card_2019 = int_player_card_framed_2019['prospect_player']
    int_player_card_2019
    int_player_report_and_video_2019 = int_player_card_2019['content']
    int_player_report_2019 = int_player_report_and_video_2019['default']
    int_card_list_2019.append(int_player_report_2019)



int_report_trim_1_list_2019 = []
for i in int_card_list_2019:
    trim = i.rfind('Overall: ') + 10
    report_trim_1 = i[trim:]
    int_report_trim_1_list_2019.append(report_trim_1)

int_report_cleaner_list_2019 = []
for i in int_report_trim_1_list_2019:
    if i.find('</a>') == -1:
        start = i.find('n<p>') + 3
    else:
        start = i.find('</a>') + 8
    trim_a = i[start:]
    int_report_cleaner_list_2019.append(trim_a)

int_reports_2019 = []
for i in int_report_cleaner_list_2019:
    without_p = i.replace('<p>', '')
    without_p2 = without_p.replace('p>', '')
    without_n = without_p2.replace('\n', '')
    lowered = without_n.lower()
    int_reports_2019.append(lowered)

int_player_id_dataframe_2019['Written Report'] = int_reports_2019
int_top_one_hundred_data_2019 = int_player_id_dataframe_2019


full_int_2019 = []

for i in int_reports_2019:
    
    int_text_2019 = TextBlob(i)
   
    int_score_2019 = int_text_2019.sentiment
    
    full_int_2019.append(int_score_2019)


int_top_one_hundred_data_2019['Sentiment Score'] = full_int_2019

sorted_int_2019 = int_top_one_hundred_data_2019.sort_values(by=['Sentiment Score'],ascending = False)
sorted_int_2019 = sorted_int_2019.drop(['Written Report', 'Player ID'], axis = 1)
sorted_int_2019

,First Name,Last Name,Position,Player Rank,Sentiment Score
21,Brayan,Medina,RHP,22,"(0.5530303030303031, 0.5212121212121213)"
18,Andry,Lara,RHP,19,"(0.3939393939393939, 0.4575757575757576)"
10,Arol,Vera,SS,11,"(0.3457142857142857, 0.47960317460317464)"
0,Jasson,Dominguez,OF,1,"(0.332010582010582, 0.5000440917107584)"
29,Rikelvin,De Castro,SS,30,"(0.2804761904761905, 0.43023809523809525)"
13,Adael,Amador,SS,14,"(0.26916666666666667, 0.48166666666666663)"
16,Dauri,Lorenzo,SS,17,"(0.26233897681266105, 0.4413630229419703)"
5,Luis,Rodriguez,OF,6,"(0.24255291005291005, 0.4615476190476191)"
24,Reginald,Preciado,SS,25,"(0.23152173913043483, 0.45678053830227744)"
23,Kristian,Cardozo,RHP,24,"(0.21508295625942686, 0.35407239819004527)"


In [8]:
#The 2018 MLB Int Class

mlb_json_id_url = 'http://m.mlb.com/gen/players/prospects/2018/playerProspects.json'
response = requests.get(mlb_json_id_url)
response.status_code
all_json = response.json()
int_prospect_list_2018 = all_json['prospect_players']['int']
int_player_ids_2018 = []
int_player_ranks_2018 = []
int_player_first_name_2018 = []
int_player_last_name_2018 = []
int_player_position_2018 = []
for i in int_prospect_list_2018:
    int_player_id_2018 = str(i['player_id'])
    int_player_ids_2018.append(int_player_id_2018)                      
    int_rank_2018 = i["rank"]
    int_player_ranks_2018.append(int_rank_2018)
    int_first_name_2018 = i["player_first_name"]
    int_player_first_name_2018.append(int_first_name_2018)
    int_last_name_2018 = i["player_last_name"]
    int_player_last_name_2018.append(int_last_name_2018)
    int_position_2018 = i["position"]
    int_player_position_2018.append(int_position_2018)
    int_player_id_data_2018 = {'First Name' : int_player_first_name_2018, 'Last Name' : int_player_last_name_2018, 'Player ID' : int_player_ids_2018, 'Position' : int_player_position_2018, 'Player Rank' : int_player_ranks_2018}

int_player_id_dataframe_2018 = pd.DataFrame(int_player_id_data_2018)


###Part 3: Scraping every Top100 report into dataframes
pipeline_url = "http://m.mlb.com/gen/players/prospects/2018/"
int_prospect_url_list_2018 = []
link_status_2018 = []
#making links
for i in int_player_ids_2018:
    prospect_url = pipeline_url + i + '.json'
    int_prospect_url_list_2018.append(prospect_url)

#testing links
for i in int_prospect_url_list_2018:    
    response = requests.get(i)
    if response.status_code == 200:
        link = 'Link is working'
    else:
        link = 'Link does not work'
    link_status_2018.append(link)
    
#making initial list of player scout-reports/cards
int_card_list_2018 = []
for i in int_prospect_url_list_2018:    
    response = requests.get(i)
    int_player_card_framed_2018 = response.json()
    int_player_card_2018 = int_player_card_framed_2018['prospect_player']
    int_player_card_2018
    int_player_report_and_video_2018 = int_player_card_2018['content']
    int_player_report_2018 = int_player_report_and_video_2018['default']
    int_card_list_2018.append(int_player_report_2018)



int_report_trim_1_list_2018 = []
for i in int_card_list_2018:
    trim = i.rfind('Overall: ') + 10
    report_trim_1 = i[trim:]
    int_report_trim_1_list_2018.append(report_trim_1)

int_report_cleaner_list_2018 = []
for i in int_report_trim_1_list_2018:
    if i.find('</a>') == -1:
        start = i.find('n<p>') + 3
    else:
        start = i.find('</a>') + 8
    trim_a = i[start:]
    int_report_cleaner_list_2018.append(trim_a)

int_reports_2018 = []
for i in int_report_cleaner_list_2018:
    without_p = i.replace('<p>', '')
    without_p2 = without_p.replace('p>', '')
    without_n = without_p2.replace('\n', '')
    lowered = without_n.lower()
    int_reports_2018.append(lowered)

int_player_id_dataframe_2018['Written Report'] = int_reports_2018
int_top_one_hundred_data_2018 = int_player_id_dataframe_2018


full_int_2018 = []

for i in int_reports_2018:
    
    int_text_2018 = TextBlob(i)
   
    int_score_2018 = int_text_2018.sentiment
    
    full_int_2018.append(int_score_2018)


int_top_one_hundred_data_2018['Sentiment Score'] = full_int_2018

sorted_int_2018 = int_top_one_hundred_data_2018.sort_values(by=['Sentiment Score'],ascending = False)
sorted_int_2018 = sorted_int_2018.drop(['Written Report', 'Player ID'], axis = 1)
sorted_int_2018

,First Name,Last Name,Position,Player Rank,Sentiment Score
11,Starlyn,Castillo,P,12,"(0.4152777777777778, 0.43472222222222223)"
14,Antonio,Gomez,C,15,"(0.3639136904761905, 0.6273065476190476)"
26,Eduardo,Lopez,OF,27,"(0.32868480725623583, 0.4247165532879818)"
7,Richard,Gallardo,P,8,"(0.32180451127819554, 0.42857142857142855)"
8,Gabriel,Rodriguez,SS,9,"(0.3027680652680653, 0.46719114219114216)"
21,Eduarqui,Fernandez,OF,22,"(0.2739583333333333, 0.4401041666666667)"
25,Alejandro,Pie,SS,26,"(0.2383861236802413, 0.44864792070674425)"
2,Diego,Cartaya,C,3,"(0.23145833333333332, 0.5189583333333333)"
1,Yennier,Cano,RHP,2,"(0.21884920634920635, 0.3301587301587302)"
10,Alexander,Vargas,SS,11,"(0.2116108171663727, 0.378071722516167)"


In [9]:
#The 2017 MLB Int Class

mlb_json_id_url = 'http://m.mlb.com/gen/players/prospects/2017/playerProspects.json'
response = requests.get(mlb_json_id_url)
response.status_code
all_json = response.json()
int_prospect_list_2017 = all_json['prospect_players']['int']
int_player_ids_2017 = []
int_player_ranks_2017 = []
int_player_first_name_2017 = []
int_player_last_name_2017 = []
int_player_position_2017 = []
for i in int_prospect_list_2017:
    int_player_id_2017 = str(i['player_id'])
    int_player_ids_2017.append(int_player_id_2017)                      
    int_rank_2017 = i["rank"]
    int_player_ranks_2017.append(int_rank_2017)
    int_first_name_2017 = i["player_first_name"]
    int_player_first_name_2017.append(int_first_name_2017)
    int_last_name_2017 = i["player_last_name"]
    int_player_last_name_2017.append(int_last_name_2017)
    int_position_2017 = i["position"]
    int_player_position_2017.append(int_position_2017)
    int_player_id_data_2017 = {'First Name' : int_player_first_name_2017, 'Last Name' : int_player_last_name_2017, 'Player ID' : int_player_ids_2017, 'Position' : int_player_position_2017, 'Player Rank' : int_player_ranks_2017}

int_player_id_dataframe_2017 = pd.DataFrame(int_player_id_data_2017)


###Part 3: Scraping every Top100 report into dataframes
pipeline_url = "http://m.mlb.com/gen/players/prospects/2017/"
int_prospect_url_list_2017 = []
link_status_2017 = []
#making links
for i in int_player_ids_2017:
    prospect_url = pipeline_url + i + '.json'
    int_prospect_url_list_2017.append(prospect_url)

#testing links
for i in int_prospect_url_list_2017:    
    response = requests.get(i)
    if response.status_code == 200:
        link = 'Link is working'
    else:
        link = 'Link does not work'
    link_status_2017.append(link)
    
#making initial list of player scout-reports/cards
int_card_list_2017 = []
for i in int_prospect_url_list_2017:    
    response = requests.get(i)
    int_player_card_framed_2017 = response.json()
    int_player_card_2017 = int_player_card_framed_2017['prospect_player']
    int_player_card_2017
    int_player_report_and_video_2017 = int_player_card_2017['content']
    int_player_report_2017 = int_player_report_and_video_2017['default']
    int_card_list_2017.append(int_player_report_2017)



int_report_trim_1_list_2017 = []
for i in int_card_list_2017:
    trim = i.rfind('Overall: ') + 10
    report_trim_1 = i[trim:]
    int_report_trim_1_list_2017.append(report_trim_1)

int_report_cleaner_list_2017 = []
for i in int_report_trim_1_list_2017:
    if i.find('</a>') == -1:
        start = i.find('n<p>') + 3
    else:
        start = i.find('</a>') + 8
    trim_a = i[start:]
    int_report_cleaner_list_2017.append(trim_a)

int_reports_2017 = []
for i in int_report_cleaner_list_2017:
    without_p = i.replace('<p>', '')
    without_p2 = without_p.replace('p>', '')
    without_n = without_p2.replace('\n', '')
    lowered = without_n.lower()
    int_reports_2017.append(lowered)

int_player_id_dataframe_2017['Written Report'] = int_reports_2017
int_top_one_hundred_data_2017 = int_player_id_dataframe_2017


full_int_2017 = []

for i in int_reports_2017:
    
    int_text_2017 = TextBlob(i)
   
    int_score_2017 = int_text_2017.sentiment
    
    full_int_2017.append(int_score_2017)


int_top_one_hundred_data_2017['Sentiment Score'] = full_int_2017

sorted_int_2017 = int_top_one_hundred_data_2017.sort_values(by=['Sentiment Score'],ascending = False)
sorted_int_2017 = sorted_int_2017.drop(['Written Report', 'Player ID'], axis = 1)
sorted_int_2017

,First Name,Last Name,Position,Player Rank,Sentiment Score
16,Roberto,Chirinos,SS,17,"(0.3005555555555555, 0.47777777777777786)"
12,Ynmanol,Marinez,SS,13,"(0.30000000000000004, 0.5027777777777778)"
11,Ronny,Rojas,SS,12,"(0.2825091575091575, 0.4816849816849817)"
7,Luis,Garcia,SS,8,"(0.2788888888888889, 0.5133333333333333)"
14,Larry,Ernesto,OF,15,"(0.2513736263736264, 0.4806711145996861)"
5,Eric,Pardinho,RHP,6,"(0.24595959595959596, 0.3306060606060606)"
26,Juan,Querecuto,SS,27,"(0.23943452380952382, 0.4002976190476191)"
19,Keyber,Rodriguez,SS,20,"(0.23690476190476192, 0.32261904761904764)"
23,Alvaro,Gonzalez,SS,24,"(0.2243055555555556, 0.49305555555555564)"
4,Everson,Pereira,OF,5,"(0.2166666666666667, 0.4516666666666667)"


In [10]:
#The 2016 MLB Int Class

mlb_json_id_url = 'http://m.mlb.com/gen/players/prospects/2016/playerProspects.json'
response = requests.get(mlb_json_id_url)
response.status_code
all_json = response.json()
int_prospect_list_2016 = all_json['prospect_players']['int']
int_player_ids_2016 = []
int_player_ranks_2016 = []
int_player_first_name_2016 = []
int_player_last_name_2016 = []
int_player_position_2016 = []
for i in int_prospect_list_2016:
    int_player_id_2016 = str(i['player_id'])
    int_player_ids_2016.append(int_player_id_2016)                      
    int_rank_2016 = i["rank"]
    int_player_ranks_2016.append(int_rank_2016)
    int_first_name_2016 = i["player_first_name"]
    int_player_first_name_2016.append(int_first_name_2016)
    int_last_name_2016 = i["player_last_name"]
    int_player_last_name_2016.append(int_last_name_2016)
    int_position_2016 = i["position"]
    int_player_position_2016.append(int_position_2016)
    int_player_id_data_2016 = {'First Name' : int_player_first_name_2016, 'Last Name' : int_player_last_name_2016, 'Player ID' : int_player_ids_2016, 'Position' : int_player_position_2016, 'Player Rank' : int_player_ranks_2016}

int_player_id_dataframe_2016 = pd.DataFrame(int_player_id_data_2016)


###Part 3: Scraping every Top100 report into dataframes
pipeline_url = "http://m.mlb.com/gen/players/prospects/2016/"
int_prospect_url_list_2016 = []
link_status_2016 = []
#making links
for i in int_player_ids_2016:
    prospect_url = pipeline_url + i + '.json'
    int_prospect_url_list_2016.append(prospect_url)

#testing links
for i in int_prospect_url_list_2016:    
    response = requests.get(i)
    if response.status_code == 200:
        link = 'Link is working'
    else:
        link = 'Link does not work'
    link_status_2016.append(link)
    
#making initial list of player scout-reports/cards
int_card_list_2016 = []
for i in int_prospect_url_list_2016:    
    response = requests.get(i)
    int_player_card_framed_2016 = response.json()
    int_player_card_2016 = int_player_card_framed_2016['prospect_player']
    int_player_card_2016
    int_player_report_and_video_2016 = int_player_card_2016['content']
    int_player_report_2016 = int_player_report_and_video_2016['default']
    int_card_list_2016.append(int_player_report_2016)



int_report_trim_1_list_2016 = []
for i in int_card_list_2016:
    trim = i.rfind('Overall: ') + 10
    report_trim_1 = i[trim:]
    int_report_trim_1_list_2016.append(report_trim_1)

int_report_cleaner_list_2016 = []
for i in int_report_trim_1_list_2016:
    if i.find('</a>') == -1:
        start = i.find('n<p>') + 3
    else:
        start = i.find('</a>') + 8
    trim_a = i[start:]
    int_report_cleaner_list_2016.append(trim_a)

int_reports_2016 = []
for i in int_report_cleaner_list_2016:
    without_p = i.replace('<p>', '')
    without_p2 = without_p.replace('p>', '')
    without_n = without_p2.replace('\n', '')
    lowered = without_n.lower()
    int_reports_2016.append(lowered)

int_player_id_dataframe_2016['Written Report'] = int_reports_2016
int_top_one_hundred_data_2016 = int_player_id_dataframe_2016


full_int_2016 = []

for i in int_reports_2016:
    
    int_text_2016 = TextBlob(i)
   
    int_score_2016 = int_text_2016.sentiment
    
    full_int_2016.append(int_score_2016)


int_top_one_hundred_data_2016['Sentiment Score'] = full_int_2016

sorted_int_2016 = int_top_one_hundred_data_2016.sort_values(by=['Sentiment Score'],ascending = False)
sorted_int_2016 = sorted_int_2016.drop(['Written Report', 'Player ID'], axis = 1)
sorted_int_2016

,First Name,Last Name,Position,Player Rank,Sentiment Score
22,Francisco,Morales,RHP,23,"(0.41500000000000004, 0.4616666666666666)"
8,Luis,Almanzar,SS,9,"(0.2866666666666666, 0.5287179487179487)"
1,Kevin,Maitan,SS,2,"(0.270031746031746, 0.3847789987789988)"
25,Yunior,Severino,SS,26,"(0.2145833333333333, 0.5536458333333334)"
9,Lazaro,Armenteros,OF,10,"(0.2051587301587302, 0.44884559884559894)"
17,David,Garcia,C,18,"(0.2041666666666667, 0.4189814814814815)"
21,Jordy,Barley,SS,22,"(0.20392156862745095, 0.33137254901960794)"
11,Jorge,Ona,OF,12,"(0.19447278911564628, 0.3621598639455782)"
16,Jonathan,Machado,CF,17,"(0.19090909090909092, 0.3991341991341992)"
27,Yerdeluis,Vargas,SS,28,"(0.18878873771730914, 0.47246409674981105)"


In [11]:
#The 2014 MLB Int Class

mlb_json_id_url = 'http://m.mlb.com/gen/players/prospects/2014/playerProspects.json'
response = requests.get(mlb_json_id_url)
response.status_code
all_json = response.json()
int_prospect_list_2014 = all_json['prospect_players']['int']
int_player_ids_2014 = []
int_player_ranks_2014 = []
int_player_first_name_2014 = []
int_player_last_name_2014 = []
int_player_position_2014 = []
for i in int_prospect_list_2014:
    int_player_id_2014 = str(i['player_id'])
    int_player_ids_2014.append(int_player_id_2014)                      
    int_rank_2014 = i["rank"]
    int_player_ranks_2014.append(int_rank_2014)
    int_first_name_2014 = i["player_first_name"]
    int_player_first_name_2014.append(int_first_name_2014)
    int_last_name_2014 = i["player_last_name"]
    int_player_last_name_2014.append(int_last_name_2014)
    int_position_2014 = i["position"]
    int_player_position_2014.append(int_position_2014)
    int_player_id_data_2014 = {'First Name' : int_player_first_name_2014, 'Last Name' : int_player_last_name_2014, 'Player ID' : int_player_ids_2014, 'Position' : int_player_position_2014, 'Player Rank' : int_player_ranks_2014}

int_player_id_dataframe_2014 = pd.DataFrame(int_player_id_data_2014)


###Part 3: Scraping every Top100 report into dataframes
pipeline_url = "http://m.mlb.com/gen/players/prospects/2014/"
int_prospect_url_list_2014 = []
link_status_2014 = []
#making links
for i in int_player_ids_2014:
    prospect_url = pipeline_url + i + '.json'
    int_prospect_url_list_2014.append(prospect_url)

#testing links
for i in int_prospect_url_list_2014:    
    response = requests.get(i)
    if response.status_code == 200:
        link = 'Link is working'
    else:
        link = 'Link does not work'
    link_status_2014.append(link)
    
#making initial list of player scout-reports/cards
int_card_list_2014 = []
for i in int_prospect_url_list_2014:    
    response = requests.get(i)
    int_player_card_framed_2014 = response.json()
    int_player_card_2014 = int_player_card_framed_2014['prospect_player']
    int_player_card_2014
    int_player_report_and_video_2014 = int_player_card_2014['content']
    int_player_report_2014 = int_player_report_and_video_2014['default']
    int_card_list_2014.append(int_player_report_2014)



int_report_trim_1_list_2014 = []
for i in int_card_list_2014:
    trim = i.rfind('Overall: ') + 10
    report_trim_1 = i[trim:]
    int_report_trim_1_list_2014.append(report_trim_1)

int_report_cleaner_list_2014 = []
for i in int_report_trim_1_list_2014:
    if i.find('</a>') == -1:
        start = i.find('n<p>') + 3
    else:
        start = i.find('</a>') + 8
    trim_a = i[start:]
    int_report_cleaner_list_2014.append(trim_a)

int_reports_2014 = []
for i in int_report_cleaner_list_2014:
    without_p = i.replace('<p>', '')
    without_p2 = without_p.replace('p>', '')
    without_n = without_p2.replace('\n', '')
    lowered = without_n.lower()
    int_reports_2014.append(lowered)

int_player_id_dataframe_2014['Written Report'] = int_reports_2014
int_top_one_hundred_data_2014 = int_player_id_dataframe_2014


full_int_2014 = []

for i in int_reports_2014:
    
    int_text_2014 = TextBlob(i)
   
    int_score_2014 = int_text_2014.sentiment
    
    full_int_2014.append(int_score_2014)


int_top_one_hundred_data_2014['Sentiment Score'] = full_int_2014

sorted_int_2014 = int_top_one_hundred_data_2014.sort_values(by=['Sentiment Score'],ascending = False)
sorted_int_2014 = sorted_int_2014.drop(['Written Report', 'Player ID'], axis = 1)
sorted_int_2014

,First Name,Last Name,Position,Player Rank,Sentiment Score
19,Ronny,Rafael,OF,20,"(0.41739316239316243, 0.5591025641025641)"
6,Jonathan,Amundaray,OF,7,"(0.3477048992673993, 0.5899324633699634)"
26,Kenny,Hernandez,SS,27,"(0.29972222222222217, 0.3705555555555556)"
5,Christopher,Acosta,RHP,6,"(0.2826007326007326, 0.3430402930402931)"
16,Huascar,Ynoa,RHP,17,"(0.2808403361344538, 0.3683193277310924)"
28,Bryan,Emery,OF,29,"(0.24684237331296152, 0.42389228418640185)"
23,Ricky,Aracena,SS,24,"(0.23657407407407405, 0.49074074074074076)"
11,Pedro,Gonzalez,SS,12,"(0.2258503401360544, 0.32329931972789117)"
7,Brayan,Hernandez,OF,8,"(0.2065934065934066, 0.421632854325162)"
24,Miguel,Flames,C,25,"(0.20541958041958042, 0.48158508158508156)"


In [12]:
#The 2013 MLB Int Class

mlb_json_id_url = 'http://m.mlb.com/gen/players/prospects/2013/playerProspects.json'
response = requests.get(mlb_json_id_url)
response.status_code
all_json = response.json()
int_prospect_list_2013 = all_json['prospect_players']['int']
int_player_ids_2013 = []
int_player_ranks_2013 = []
int_player_first_name_2013 = []
int_player_last_name_2013 = []
int_player_position_2013 = []
for i in int_prospect_list_2013:
    int_player_id_2013 = str(i['player_id'])
    int_player_ids_2013.append(int_player_id_2013)                      
    int_rank_2013 = i["rank"]
    int_player_ranks_2013.append(int_rank_2013)
    int_first_name_2013 = i["player_first_name"]
    int_player_first_name_2013.append(int_first_name_2013)
    int_last_name_2013 = i["player_last_name"]
    int_player_last_name_2013.append(int_last_name_2013)
    int_position_2013 = i["position"]
    int_player_position_2013.append(int_position_2013)
    int_player_id_data_2013 = {'First Name' : int_player_first_name_2013, 'Last Name' : int_player_last_name_2013, 'Player ID' : int_player_ids_2013, 'Position' : int_player_position_2013, 'Player Rank' : int_player_ranks_2013}

int_player_id_dataframe_2013 = pd.DataFrame(int_player_id_data_2013)


###Part 3: Scraping every Top100 report into dataframes
pipeline_url = "http://m.mlb.com/gen/players/prospects/2013/"
int_prospect_url_list_2013 = []
link_status_2013 = []
#making links
for i in int_player_ids_2013:
    prospect_url = pipeline_url + i + '.json'
    int_prospect_url_list_2013.append(prospect_url)

#testing links
for i in int_prospect_url_list_2013:    
    response = requests.get(i)
    if response.status_code == 200:
        link = 'Link is working'
    else:
        link = 'Link does not work'
    link_status_2013.append(link)
    
#making initial list of player scout-reports/cards
int_card_list_2013 = []
for i in int_prospect_url_list_2013:    
    response = requests.get(i)
    int_player_card_framed_2013 = response.json()
    int_player_card_2013 = int_player_card_framed_2013['prospect_player']
    int_player_card_2013
    int_player_report_and_video_2013 = int_player_card_2013['content']
    int_player_report_2013 = int_player_report_and_video_2013['default']
    int_card_list_2013.append(int_player_report_2013)



int_report_trim_1_list_2013 = []
for i in int_card_list_2013:
    trim = i.rfind('Overall: ') + 10
    report_trim_1 = i[trim:]
    int_report_trim_1_list_2013.append(report_trim_1)

int_report_cleaner_list_2013 = []
for i in int_report_trim_1_list_2013:
    if i.find('</a>') == -1:
        start = i.find('n<p>') + 3
    else:
        start = i.find('</a>') + 8
    trim_a = i[start:]
    int_report_cleaner_list_2013.append(trim_a)

int_reports_2013 = []
for i in int_report_cleaner_list_2013:
    without_p = i.replace('<p>', '')
    without_p2 = without_p.replace('p>', '')
    without_n = without_p2.replace('\n', '')
    lowered = without_n.lower()
    int_reports_2013.append(lowered)

int_player_id_dataframe_2013['Written Report'] = int_reports_2013
int_top_one_hundred_data_2013 = int_player_id_dataframe_2013


full_int_2013 = []

for i in int_reports_2013:
    
    int_text_2013 = TextBlob(i)
   
    int_score_2013 = int_text_2013.sentiment
    
    full_int_2013.append(int_score_2013)


int_top_one_hundred_data_2013['Sentiment Score'] = full_int_2013

sorted_int_2013 = int_top_one_hundred_data_2013.sort_values(by=['Sentiment Score'],ascending = False)
sorted_int_2013 = sorted_int_2013.drop(['Written Report', 'Player ID'], axis = 1)
sorted_int_2013

,First Name,Last Name,Position,Player Rank,Sentiment Score
2,Gleyber,Torres,SS,3,"(0.4815873015873016, 0.5908730158730159)"
27,Nicolas Pierre,Figueroa,OF,28,"(0.4, 0.38)"
4,Leonardo,Molina,OF,5,"(0.34453125, 0.4994791666666667)"
14,Carlos,Herrera,SS,15,"(0.33814814814814814, 0.5174358974358975)"
3,Marten,Gasparini,SS,4,"(0.32678571428571423, 0.5142857142857142)"
13,Erick,Julio,RHP,14,"(0.3138675213675214, 0.43201923076923077)"
1,Micker Adolfo,Zapata,OF,2,"(0.30128205128205127, 0.40320512820512816)"
15,Anderson,Franco,3B,16,"(0.26173469387755105, 0.48835034013605444)"
8,Mayky,Perez,RHP,9,"(0.2547494172494172, 0.3593240093240093)"
20,Bryan,Lizardo,3B,21,"(0.24817460317460316, 0.40055555555555566)"
